In [58]:
# import libraries 
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

In [15]:
# calculate average size to help inform a size to resize all images to
imagex = 0
imagey = 0
for i in range(1, 5489):    
    image = Image.open(f'Data/train/img_00{i:04d}.jpg')
    imagex += image.size[0]
    imagey += image.size[1]
    
print(imagex/5488) # 50.567237609329446
print(imagey/5488) # 50.05648688046647

# scale to 50 x 50
rescale_size = (50, 50)

50.567237609329446
50.05648688046647


In [ ]:
# import image files as pixel RGB values

train_data = []
train_labels = []

# import paths and labels
train_metadata = pd.read_csv("Data/train/train_metadata.csv", index_col="id")
train_paths = train_metadata["image_path"]
train_class = train_metadata["ClassId"]

for i in range(len(train_paths)):
    try:
        # open image
        image = Image.open(f"Data/train/{train_paths.iloc[i]}")
        # resize image to (50, 50)
        image = image.resize(rescale_size)
        image = np.array(image)
        train_data.append(image)
        train_labels.append(train_class.iloc[i])
    except:
        print(f"error loading image {train_paths.iloc[i]}")

data = np.array(train_data)
labels = np.array(train_labels)

array([[[[ 81,  70,  71],
         [ 79,  69,  71],
         [ 74,  65,  68],
         ...,
         [ 71,  61,  60],
         [ 66,  59,  58],
         [ 61,  57,  56]],

        [[ 79,  68,  70],
         [ 77,  67,  69],
         [ 73,  64,  67],
         ...,
         [ 70,  61,  60],
         [ 66,  60,  58],
         [ 61,  58,  56]],

        [[ 77,  66,  69],
         [ 75,  65,  67],
         [ 72,  63,  65],
         ...,
         [ 68,  61,  59],
         [ 65,  61,  58],
         [ 62,  60,  57]],

        ...,

        [[ 35,  35,  33],
         [ 35,  35,  32],
         [ 36,  35,  31],
         ...,
         [ 43,  37,  34],
         [ 42,  37,  35],
         [ 42,  37,  35]],

        [[ 33,  32,  32],
         [ 33,  32,  31],
         [ 34,  33,  31],
         ...,
         [ 39,  34,  33],
         [ 38,  34,  33],
         [ 38,  34,  33]],

        [[ 32,  31,  31],
         [ 33,  31,  31],
         [ 34,  32,  32],
         ...,
         [ 39,  34,  34],
        

In [ ]:
# sanity check data

print(f"data has shape: {data.shape}")
print(f"labels have shape: {labels.shape}")

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=69)

print(f"X_train has shape: {X_train.shape}")
print(f"X_test has shape: {X_test.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"y_test has shape: {y_test.shape}")

y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

data has shape: (5488, 50, 50, 3)
labels have shape: (5488,)
X_train has shape: (4390, 50, 50, 3)
X_test has shape: (1098, 50, 50, 3)
y_train has shape: (4390,)
y_test has shape: (1098,)
(4390, 43)


In [53]:
# Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.15))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.20))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(43, activation='softmax'))

# Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#M odel display
model.summary() 

C:\Users\rayzh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 42, 42, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 19, 19, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 17, 17, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     8,389,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        22,059 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,833,899 (33.70 MB)

 Trainable params: 8,833,899 (33.70 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# Training the Model
with tf.device('/GPU:0'):
    epochs = 35
    history1 = model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - accuracy: 0.0855 - loss: 14.9181 - val_accuracy: 0.4299 - val_loss: 2.1528
Epoch 2/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 205ms/step - accuracy: 0.5303 - loss: 1.8151 - val_accuracy: 0.8215 - val_loss: 0.8276
Epoch 3/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 204ms/step - accuracy: 0.8068 - loss: 0.6909 - val_accuracy: 0.9308 - val_loss: 0.3511
Epoch 4/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 208ms/step - accuracy: 0.9052 - loss: 0.3525 - val_accuracy: 0.9353 - val_loss: 0.2423
Epoch 5/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 207ms/step - accuracy: 0.9348 - loss: 0.2465 - val_accuracy: 0.9472 - val_loss: 0.2105
Epoch 6/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 211ms/step - accuracy: 0.9421 - loss: 0.2191 - val_accuracy: 0.9599 - val_loss: 0.1975
Epoch 7/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 209ms/step - accuracy: 0.9605 - loss: 0.1597 - val_accuracy: 0.9545 - val_loss: 0.1896
Epoch 8/35
35/35 ━━━━━━━━━━━━━━━━━━━━ 8s 219ms/step - accuracy: 0.9643 - loss: 0.1286 - val_accuracy: 0

In [ ]:
# import test set
test_metadata = pd.read_csv("Data/test/test_metadata.csv")
test_paths = test_metadata["image_path"]
test_index = test_metadata["id"]

test_data = []
test_id = []

for i in range(len(test_paths)):
    try:
        # open image
        image = Image.open(f"Data/test/{test_paths.iloc[i]}")
        # resize image to (50, 50)
        image = image.resize(rescale_size)
        image = np.array(image)
        test_data.append(image)
        test_id.append(test_index.iloc[i])
    except:
        print(f"error loading image {test_paths.iloc[i]}")

test_data = np.array(test_data)

print(test_data.shape)
print(test_id)


(2353, 50, 50, 3)
[661, 4477, 1046, 631, 6533, 2899, 1941, 5749, 588, 1333, 5826, 7493, 4306, 3429, 1191, 4076, 1601, 2183, 4214, 5366, 5434, 6709, 2421, 4247, 5722, 1115, 1730, 6673, 156, 2011, 4416, 4661, 6511, 4411, 821, 6246, 3934, 5711, 3233, 3738, 4561, 3780, 6288, 3115, 7155, 7603, 1932, 6387, 6001, 4376, 2393, 5025, 660, 7634, 1978, 2398, 2826, 2679, 1466, 1953, 3887, 5681, 2945, 2481, 6552, 6096, 1969, 5544, 133, 2313, 5148, 3875, 1050, 960, 721, 2129, 6395, 4106, 1809, 1882, 6491, 7103, 2490, 2810, 3378, 2922, 269, 4468, 1162, 6523, 2106, 5505, 3292, 6605, 3836, 1188, 5752, 2627, 3829, 3616, 1085, 214, 5685, 5261, 879, 4070, 5183, 459, 5777, 4532, 3973, 6037, 2278, 2687, 1388, 6664, 966, 731, 1808, 7703, 5302, 5622, 2637, 5146, 3617, 5720, 5260, 7490, 2428, 7268, 6915, 5690, 6363, 3527, 4800, 1167, 3993, 93, 7816, 1116, 6634, 6411, 6616, 6299, 1866, 7400, 4527, 1528, 7801, 218, 3487, 6009, 1686, 4248, 5258, 3236, 5489, 6993, 6925, 3221, 7668, 4051, 4200, 5321, 5835, 3305, 562

In [ ]:
# predict classes of test data
with tf.device('/GPU:0'):
    pred = np.argmax(model.predict(test_data), axis=-1)

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


In [82]:
result = pd.DataFrame({'id': test_index, 'ClassId': pred})
result.to_csv("results.csv", index=False)